# THE MOMENT WE'VE BEEN WAITING FOR!!!

### We're doing evolutionary stuff ~*

In [1]:
import os
import sys
import logging
FORMAT = "%(filename)s:%(lineno)d %(funcName)s %(levelname)s %(message)s"
logging.basicConfig(format=FORMAT, level=logging.INFO)

import re
import imp
import itertools
import random
import numpy as np
from numpy import array
import pandas as pd
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns


# do this before we have a chance to import openbabel!
import rdkit, rdkit.Chem.rdDistGeom, rdkit.DistanceGeometry

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import rdBase

import py3Dmol

from rmgpy.molecule import Molecule
from rmgpy.species import Species
from rmgpy.reaction import Reaction


from multi_molecule import *
from multi_reaction import *

from ase.calculators.morse import * #chosing this calculator for now because it's fast
from ase.calculators.dftb import *
from ase.calculators.lj import *
from ase.calculators.emt import *

from copy import deepcopy

/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
thermo.py:760 loadLibraries INFO Loading thermodynamics library from CBS_QB3_1dHR.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:760 loadLibraries INFO Loading thermodynamics library from KlippensteinH2O2.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:760 loadLibraries INFO Loading thermodynamics library from primaryThermoLibrary.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:760 loadLibraries INFO Loading thermodynamics library from thermo_DFT_CCSDTF12_BAC.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:774 loadGroups INFO Loading thermodynamics group database from /Users/nathan/Code/RMG-database/input/

In [2]:
mol = Multi_Molecule("COCCC")
mol.ase_molecule.set_calculator(EMT())
mol.view_mol()

In [3]:
possible_dihedrals = np.arange(0, 360+30, 30)


population_size = 200

population = []

for i in range(population_size):
    mol_copy = deepcopy(mol)
    dihedrals = []
    
    for torsion in mol_copy.torsions:
        dihedral = np.random.choice(possible_dihedrals)
        dihedrals.append(dihedral)
        i,j,k,l = torsion.indices
        RHS = torsion.RHS
        
        mol_copy.ase_molecule.set_dihedral(i,j,k,l, dihedral)
    
    mol_copy.update_geometry_from_ase_mol()
    
    
    e = mol_copy.ase_molecule.get_potential_energy()
    
    population.append( [e] + dihedrals )
    
df = pd.DataFrame(population)
columns = ["Energy"]
for i in range(len(mol.torsion_list)):
    columns = columns +["Torsion " + str(i)] 
df.columns = columns
df = df.sort("Energy")
df

/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:32: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,Energy,Torsion 0,Torsion 1
187,5.391402,150,210
143,5.391402,150,210
99,5.434459,90,120
89,5.434459,90,120
145,5.434459,90,120
2,5.643340,120,120
54,5.643340,120,120
134,5.643340,120,120
179,5.785614,150,240
175,5.915168,120,150


In [4]:
top_percent = 0.30

top_population = population_size * top_percent

top = df.iloc[:int(top_population),:]

In [5]:
results = []
for i in range(population_size):
    dihedrals = []
    for j, torsion in enumerate(mol.torsions):
        
        i,j,k,l = torsion.indices
        RHS = torsion.RHS
        
        dihedral = random.gauss(top.mean()[j+1], top.std()[j+1])
        dihedrals.append(dihedral)
        mol.ase_molecule.set_dihedral(a1 = i,
                                a2 = j, 
                                a3 = k, 
                                a4 = l, 
                                angle= float(dihedral), 
                                indices=RHS)
        
    e = mol.ase_molecule.get_potential_energy()
    results.append([e] + dihedrals)
top = df.iloc[:int(top_population),:]
    
df = pd.DataFrame(results)

columns = ["Energy"]
for i in range(len(mol.torsion_list)):
    columns = columns +["Torsion " + str(i)] 
    
df.columns = columns
df = df.sort("Energy")
df
    
        
        
        

/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:29: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,Energy,Torsion 0,Torsion 1
98,4.916553,54.102533,51.019371
131,4.919974,56.308467,46.276533
185,4.921461,44.972896,400.718444
151,4.921953,-12.794115,230.192372
152,4.926786,54.946599,258.161346
35,4.927697,67.339610,269.843497
48,4.929366,85.285681,58.455190
45,4.930593,149.824080,406.951832
172,4.931130,95.470919,271.929268
28,4.932010,118.356291,303.914266


In [6]:
generations = 20
q = 0
while (top.std()[1] > 0.01) and (top.std()[2] > 0.01):
    
    print "This is the {}th generation".format(q)
    q = q + 1
    

    results = []
    for i in range(population_size):
        dihedrals = []
        for j, torsion in enumerate(mol.torsions):

            i,j,k,l = torsion.indices
            RHS = torsion.RHS

            dihedral = random.gauss(top.mean()[j+1], top.std()[j+1])
            dihedrals.append(dihedral)
            mol.ase_molecule.set_dihedral(a1 = i,
                                    a2 = j, 
                                    a3 = k, 
                                    a4 = l, 
                                    angle= float(dihedral), 
                                    indices=RHS)

        e = mol.ase_molecule.get_potential_energy()
        results.append([e] + dihedrals)

    df = pd.DataFrame(results)

    columns = ["Energy"]
    for i in range(len(mol.torsion_list)):
        columns = columns +["Torsion " + str(i)] 

    df.columns = columns
    df = df.sort("Energy")
    top = df.iloc[:int(top_population),:]
df       

This is the 0th generation


/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:36: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


This is the 1th generation
This is the 2th generation
This is the 3th generation
This is the 4th generation
This is the 5th generation
This is the 6th generation
This is the 7th generation
This is the 8th generation
This is the 9th generation
This is the 10th generation
This is the 11th generation
This is the 12th generation
This is the 13th generation
This is the 14th generation
This is the 15th generation
This is the 16th generation
This is the 17th generation
This is the 18th generation
This is the 19th generation
This is the 20th generation
This is the 21th generation


,Energy,Torsion 0,Torsion 1
54,4.896840,0.759251,292.570686
152,4.896840,0.764472,292.569985
90,4.896840,0.755558,292.574011
81,4.896840,0.766401,292.569318
6,4.896840,0.759703,292.575342
16,4.896840,0.755461,292.570097
175,4.896840,0.761191,292.567676
67,4.896840,0.766839,292.572588
13,4.896840,0.750909,292.571330
115,4.896840,0.751660,292.578194


In [7]:
tor0, tor1 = mol.torsions

for i, tor in enumerate(mol.torsions):
    dihedral = df.iloc[0][i+1]
    i,j,k,l = torsion.indices
    RHS = torsion.RHS
    mol.ase_molecule.set_dihedral(a1 = i,
                                    a2 = j, 
                                    a3 = k, 
                                    a4 = l, 
                                    angle= float(dihedral), 
                                    indices=RHS)
mol.view_mol()
    

In [8]:

#mol.ase_molecule.set_calculator(calc)

# Getting the torsion combos
torsion_list = mol.torsions
torsion_angles = np.arange(0, 360,5) ### You can change the degree step size
torsion_combos = list( itertools.combinations_with_replacement( torsion_angles, len(torsion_list)) )
if len(torsion_list) != 1:
    torsion_combos = list(
        set(
            torsion_combos + 
            list(itertools.combinations_with_replacement( 
                torsion_angles[::-1], len(torsion_list)
            ))))

    results = []
    
# Calculating the potential energy for each conformation
for combo in torsion_combos:
    geo = zip(torsion_list, combo)
    #print geo
    for torsion in geo:
        tor = torsion[0]
        #print tor
        angle = torsion[1]
        
        i,j,k,l = tor.indices
        RHS =  tor.RHS
        mol.ase_molecule.set_dihedral(a1 = i,
                                a2 = j, 
                                a3 = k, 
                                a4 = l, 
                                angle= float(angle), 
                                indices=RHS)
    mol.update_geometry_from_ase_mol()
    results.append([mol.ase_molecule.get_potential_energy()] + list(combo) )

# Creating a dataframe of the results
brute_force = pd.DataFrame(results)
columns = ["Energy"]
for i in range(len(torsion_list)):
    columns = columns + ["Torsion " + str(i)] 

brute_force.columns = columns
brute_force = brute_force.sort("Energy")
brute_force

/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:45: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,Energy,Torsion 0,Torsion 1
3496,4.897309,5,290
983,4.897314,355,295
4144,4.897402,0,295
377,4.897462,0,290
425,4.897977,350,295
5071,4.898282,355,290
4663,4.898671,10,290
3851,4.898842,345,295
3612,4.899323,340,300
4673,4.899338,350,290


In [9]:

tor0, tor1 = mol.torsions

for i, tor in enumerate(mol.torsions):
    dihedral = brute_force.iloc[0][i+1]
    i,j,k,l = tor.indices
    RHS = tor.RHS
    mol.ase_molecule.set_dihedral(a1 = i,
                                    a2 = j, 
                                    a3 = k, 
                                    a4 = l, 
                                    angle= float(dihedral), 
                                    indices=RHS)
mol.view_mol()
    